## Generate cretria for each sentement and examples 

In [101]:
# Ensure your OpenAI API key is set
import pandas as pd
import json
import os

from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


os.environ["OPENAI_API_KEY"] = "your_api_key"
openai_api_key = os.getenv("OPENAI_API_KEY")

In [102]:

model_name="gpt-4o"
#gpt-3.5-turbo

temperature=0.7
max_tokens=500

In [107]:
# Read the CSV file into a DataFrame
df = pd.read_csv("ctg_1example_pol.csv")
df

,Category,Left,Center,Right
0,music,Barbra Streisand shares cartoon of Pelosi kill...,Drake Has Something to Say to the ''Kids'' Who...,Music education is crucial for fostering creat...
1,basketball,Baltimore Hits 300 Homicides For 5th Consecuti...,Basketball programs in schools play a crucial ...,Trump welcomes Washington Nationals to White H...
2,movies,'My 82-year-old bones hurt': Jane Fonda spent ...,Billie Lourd reflects on mom Carrie Fisher and...,Family films promoting traditional values are ...
3,baseball,Can James Wiseman and Memphis topple the NCAA ...,The College Hoops Contender Series: Who are th...,Baseball’s success highlights the benefits of ...
4,lifestyle,1619 Project reporter details series about 400...,Thousands of animals around the world are at r...,Self-reliance in personal finance is key to a ...
5,weather,Marginalized communities often bear the brunt ...,"California governor ""owns"" blackouts, takes po...",Local communities must take initiative in weat...
6,travel,Mayor de Blasio introduces Outreach NYC. The d...,Kroger shooting witness' car impounded as evid...,Traveling within the country boosts local econ...
7,autos,D.C. empowers bike lane enforcement officers t...,One of the Most Controversial F1 Crashes Ever ...,Buying American-made vehicles supports domesti...
8,scienceandtechnology,Mediterranean basin badly hit by climate chang...,A Google Fitbit means new possibilities and qu...,Tim Cook to host Donald Trump at Texas Apple f...
9,morenews,Universal healthcare is essential for a just s...,Pete Buttigieg clarifies views on the Obama ad...,Strong national security policies are essentia...


In [108]:
# Function to generate criteria string for a given category
def generate_criteria(category):
    
    prompt_template = """
   Generate detailed criteria for determining the political lean (left, center, or right) of {category} news articles. 
   Provide comprehensive and specific criteria for identifying left, center, 
   or right political lean within the context of {category} news category. 
   Include content types that typically correspond to each sentiment classification.
   
   Your response only has the description of each political lean.
   Please provide the response without using any formatting symbols like asterisks or hashtags.    

   
    
    Left:
    

    Center:
    

    Right:
    
    """

        # Create the prompt template
    prompt = PromptTemplate(input_variables=["category"], template=prompt_template)

    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(model_name=model_name, openai_api_key=openai_api_key, temperature=temperature, max_tokens=max_tokens)
    # Create the LLMChain with the prompt and the LLM
    chain = LLMChain(prompt=prompt, llm=llm)
    result = chain.run(category=category)
    return result.strip()



In [109]:

def generate_CoTExamples(category, criteria_string, left, center, right):
    
    prompt_template ="""
    You will be provided with three news articles about {category}. 
    Your role is to determine the political lean of the articles. 
    Use the following this step-by-step approach:
    
    1. Identify key points in the article.
    2. Determine political lean indicators in the article based on the criteria: {criteria_string}
    3. Conclude the overall political lean.
    
    Your response should have the following structure. 
    Please provide the response without using any formatting symbols like asterisks or hashtags.    
    
    Article 1: {left}
    1. Key points: 
    2. Political lean indicators:
    3. Political lean:

    Article 2: {center}
    1. Key points: 
    2. Political lean indicators:
    3. Political lean:
    
    
    Article 3: {right}
    1. Key points: 
    2. Political lean indicators:
    3. Political lean:
    
    
    """
    # Create the prompt template
    prompt = PromptTemplate(input_variables=["category", "criteria_string", "left", "center", "right"], template=prompt_template)
    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(model_name=model_name, openai_api_key=openai_api_key, temperature=0.2, max_tokens=max_tokens)
    # Create the LLMChain with the prompt and the LLM
    chain = LLMChain(prompt=prompt, llm=llm)
    result = chain.run(category=category, criteria_string=criteria_string, left=left, center=center, right=right)
    return result.strip()



In [110]:
def parse_criteria(combined_string):
    criteria = {"left": [], "center": [], "right": []}
    examples = []
    current_section = None

    for line in combined_string.split('\n'):
        line = line.strip()
        if line.startswith("Left:"):
            current_section = "left"
        elif line.startswith("Center:"):
            current_section = "center"
        elif line.startswith("Right:"):
            current_section = "right"
        elif line.startswith("Article"):
            current_section = "articles"
            examples.append(line)  # Start a new article
        elif current_section and line:
            if current_section == "articles":
                examples.append(line)
            else:
                criteria[current_section].append(line)

    return criteria, "\n".join(examples)




# Generate criteria for each category and format them into a configuration


categories = df['Category'].tolist()
left = df['Left'].tolist()
center = df['Center'].tolist()
right = df['Right'].tolist()

for i in range(len(categories)):
    config = {}
    criteria_string = generate_criteria(categories[i])
    examples_string= generate_CoTExamples(categories[i], criteria_string, left[i], center[i], right[i])
    combined_string = criteria_string + "\n" + examples_string

    criteria, examples = parse_criteria(combined_string)
    config = {
        categories[i]: {
            "left": " ".join(criteria["left"]),
            "center": " ".join(criteria["center"]),
            "right": " ".join(criteria["right"]),
            "chain_of_thought_examples": examples.strip()
        }
    }

    # Save the configuration to a JSON file
    path= "pol_criteria_examples/"
    with open(path+categories[i]+'.json', 'w') as file:
        json.dump(config, file, indent=4)

    print("Configuration file created with generated criteria and chain of thought examples.")



Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration file created with generated criteria and chain of thought examples.
Configuration fi

## Convert JSON file to txt 

In [111]:
import json
import os
import glob

def convert_json_to_text(json_data):
    output = []

    for category, sentiments in json_data.items():
        output.append(f"{category.capitalize()}:\n")
        for sentiment, text in sentiments.items():
            text = text.replace('\\u2019', "'")
            output.append(f"{sentiment.capitalize()}:\n{text}\n")

    return "\n".join(output)



txt_path="pol_criteria_examples/txtFiles/"
json_path= "pol_criteria_examples/"


# Use glob to get all the JSON files in the directory
json_files = glob.glob(os.path.join(json_path, '*.json'))

# Loop through the JSON files
for json_file in json_files:
    with open(json_file, 'r') as file1:
        json_data = json.load(file1)

    text_output = convert_json_to_text(json_data)
    
    # Correctly form the file name by joining the directory and the file name
    file_name = os.path.join(txt_path, list(json_data.keys())[0] + ".txt")
    with open(file_name, 'w') as file2:
        file2.write(text_output)

    print(f"Conversion complete! Check the '{file_name}' file.")


Conversion complete! Check the '/home/alshma0a/PycharmProjects/Agent4Rec_v2/my notebook/pol_criteria_examples/txtFiles/video.txt' file.
Conversion complete! Check the '/home/alshma0a/PycharmProjects/Agent4Rec_v2/my notebook/pol_criteria_examples/txtFiles/entertainment.txt' file.
Conversion complete! Check the '/home/alshma0a/PycharmProjects/Agent4Rec_v2/my notebook/pol_criteria_examples/txtFiles/travel.txt' file.
Conversion complete! Check the '/home/alshma0a/PycharmProjects/Agent4Rec_v2/my notebook/pol_criteria_examples/txtFiles/finance.txt' file.
Conversion complete! Check the '/home/alshma0a/PycharmProjects/Agent4Rec_v2/my notebook/pol_criteria_examples/txtFiles/politics.txt' file.
Conversion complete! Check the '/home/alshma0a/PycharmProjects/Agent4Rec_v2/my notebook/pol_criteria_examples/txtFiles/health.txt' file.
Conversion complete! Check the '/home/alshma0a/PycharmProjects/Agent4Rec_v2/my notebook/pol_criteria_examples/txtFiles/baseball.txt' file.
Conversion complete! Check the